In [1]:
import json

import numpy as np
import pandas as pd
from money_parser import price_dec

pd.options.display.max_columns = None

In [2]:
def curr_to_float(v): return float(price_dec(v)) if v else v
def tf_to_10(v): return 1 if v == 't' else 0
def nulls_to_0(v): return float(0 if (pd.isnull(v) or v == '') else v)

converters_to_columns = [
    (curr_to_float, ['price', 'weekly_price', 'monthly_price']),
    (tf_to_10, ['instant_bookable', 'host_is_superhost']),
    (nulls_to_0, ['bedrooms', 'bathrooms'])
]

converters = {col: conv for conv, cols in converters_to_columns for col in cols}

df = pd.read_csv('data/listings.csv', index_col='id', converters=converters)

/home/abhivik/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
complaints_df = pd.read_csv('data/NYPD_Complaint_Data_Historic.csv', index_col='CMPLNT_NUM')

/home/abhivik/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/abhivik/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
cleaned_complaints_df = complaints_df[~complaints_df['Latitude'].isnull()].copy()

In [5]:
lats = pd.cut(cleaned_complaints_df['Latitude'], 30)
lons = pd.cut(cleaned_complaints_df['Longitude'], 30)

In [6]:
lats = lats.apply(lambda v: str(v.left) + ',' + str(v.right))
lons = lons.apply(lambda v: str(v.left) + ',' + str(v.right))

In [7]:
boxes = lats.str.cat(lons, sep=',')

In [8]:
cleaned_complaints_df['boxes'] = boxes

In [10]:
def convert_to_number(val):
    if (val == 'FELONY'):
        return 5
    elif (val == 'MISDEMEANOR'):
        return 1
    else:
        return 0.1

cleaned_complaints_df['LAW_CAT_CD_NUMBER'] = cleaned_complaints_df['LAW_CAT_CD'].apply(convert_to_number)

In [11]:
box_df = cleaned_complaints_df[['boxes', 'LAW_CAT_CD_NUMBER']].groupby('boxes').sum().reset_index().copy()

In [12]:
box_df['lower_latitude'] = box_df['boxes'].apply(lambda v: float(v.split(',')[0]))
box_df['upper_latitude'] = box_df['boxes'].apply(lambda v: float(v.split(',')[1]))
box_df['lower_longitude'] = box_df['boxes'].apply(lambda v: float(v.split(',')[2]))
box_df['upper_longitude'] = box_df['boxes'].apply(lambda v: float(v.split(',')[3]))

In [13]:
box_df['lower_latitude'] = box_df['lower_latitude'].apply(lambda v: 0 if v == box_df['lower_latitude'].min() else v)
box_df['upper_latitude'] = box_df['upper_latitude'].apply(lambda v: (v+1) if v == box_df['upper_latitude'].max() else v)
box_df['lower_longitude'] = box_df['lower_longitude'].apply(lambda v: (v-1) if v == box_df['lower_longitude'].min() else v)
box_df['upper_longitude'] = box_df['upper_longitude'].apply(lambda v: 0 if v == box_df['upper_longitude'].max() else v)

In [22]:
df['box'] = ''
df['box_lower_latitude'] = np.nan
df['box_upper_latitude'] = np.nan
df['box_lower_longitude'] = np.nan
df['box_upper_longitude'] = np.nan
df['crime_score'] = np.nan
done_count = 0
for index, row in df.iterrows():
    s1 = np.logical_and(row.latitude > box_df['lower_latitude'], row.latitude <= box_df['upper_latitude'])
    s2 = np.logical_and(row.longitude > box_df['lower_longitude'], row.longitude <= box_df['upper_longitude'])
    box = box_df[np.logical_and(s1, s2)]
    try:
        box_dict = box.iloc[0].to_dict()
        df.at[index, 'box'] = box_dict['boxes']
        df.at[index, 'box_lower_latitude'] = box_dict['lower_latitude']
        df.at[index, 'box_upper_latitude'] = box_dict['upper_latitude']
        df.at[index, 'box_lower_longitude'] = box_dict['lower_longitude']
        df.at[index, 'box_upper_longitude'] = box_dict['upper_longitude']
        df.at[index, 'crime_score'] = box_dict['LAW_CAT_CD_NUMBER']
    except Exception as e:
        print(e)
        display(index)
    done_count+=1
    if done_count % 5000 == 0:
        print('done ', done_count)

done  5000
done  10000
done  15000
done  20000
done  25000
done  30000
done  35000
done  40000
done  45000
done  50000


In [25]:
df[['neighbourhood_cleansed', 'crime_score']].groupby('neighbourhood_cleansed').mean().sort_values('crime_score', ascending=False)

,crime_score
neighbourhood_cleansed,
Theater District,46733.929126
Flatiron District,37010.700000
Gramercy,36418.281921
NoHo,31342.700000
Kips Bay,30769.639597
...,...
Prince's Bay,470.900000
Huguenot,419.100000
New Dorp Beach,386.200000


In [30]:
box_df = box_df.rename(columns={'LAW_CAT_CD_NUMBER', 'crime_score', 'boxes': 'box'})
box_df = box_df[['lower_latitude', 'upper_latitude', 'lower_longitude', 'upper_longitude', 'box', 'crime_score']]

In [32]:
(df[['box_lower_latitude', 'box_upper_latitude', 'box_lower_longitude', 
     'box_upper_longitude', 'box', 'crime_score']].to_csv('data/listings_with_crime_scores.csv'))

In [33]:
box_df.to_csv('data/crime_scores.csv')